应用 Function
====

要将您自己或其他库的函数应用于pandas对象，主要有以下三种方法。使用哪个方法适当，取决于函数是期望在整个`DataFrame`或`Series`上，还是行或列方式，或元素方式上运行。

1. [Tablewise Function Application](http://pandas.pydata.org/pandas-docs/version/0.20.3/basics.html#tablewise-function-application): [`pipe()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.pipe.html#pandas.DataFrame.pipe)
2. [Row or Column-wise Function Application](http://pandas.pydata.org/pandas-docs/version/0.20.3/basics.html#row-or-column-wise-function-application): [`apply()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.apply.html#pandas.DataFrame.apply)
3. [Aggregation API](http://pandas.pydata.org/pandas-docs/version/0.20.3/basics.html#aggregation-api): [`agg()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.agg.html#pandas.DataFrame.agg) and [`transform()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.transform.html#pandas.DataFrame.transform)
4. [Applying Elementwise Functions](http://pandas.pydata.org/pandas-docs/version/0.20.3/basics.html#applying-elementwise-functions): [`applymap()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.applymap.html#pandas.DataFrame.applymap)

Tablewise Function Application: pipe()
====
**表格式应用函数**

`DataFrames` and `Series` can of course just be passed into functions. However, if the function needs to be called in a chain, consider using the [`pipe()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.pipe.html#pandas.DataFrame.pipe) method. Compare the following

`DataFrames`和`Series`当然可以被传递给函数。但是，如果需要在链中调用函数，请考虑使用[`pipe()`](http://pandas.pydata.org/pandas-docs/version/0.20.3/generated/pandas.DataFrame.pipe.html#pandas.DataFrame.pipe) 方法。 比较以下内容：

```
# f, g, and h are functions taking and returning ``DataFrames``
>>> f(g(h(df), arg1=1), arg2=2, arg3=3)
```

下面的代码与此等价：

```
>>> (df.pipe(h)
       .pipe(g, arg1=1)
       .pipe(f, arg2=2, arg3=3)
    )
    ```

Pandas encourages the second style, which is known as method chaining. `pipe` makes it easy to use your own or another library’s functions in method chains, alongside pandas’ methods.

pandas鼓励第二种风格，即所谓的方法链。 `pipe`可以很容易地在方法链中使用自己定义的或其他库的函数，以及pandas的方法。 

In the example above, the functions `f`, `g`, and `h` each expected the `DataFrame` as the first positional argument. What if the function you wish to apply takes its data as, say, the second argument? In this case, provide `pipe` with a tuple of `(callable, data_keyword)`. `.pipe` will route the `DataFrame` to the argument specified in the tuple.

在上面的例子中，函数`f`，`g`和`h`都将`DataFrame`作为第一个位置参数。 如果您要应用的函数将其数据作为第二个参数，该怎么办？ 在这种情况下，为`pipe`提供一个`（callable，data_keyword）`元组。 `.pipe`将`DataFrame`路由到元组中指定的参数。

For example, we can fit a regression using statsmodels. Their API expects a formula first and a `DataFrame` as the second argument, `data`. We pass in the function, keyword pair `(sm.poisson, 'data')` to `pipe`:

例如，我们可以使用statsmodels进行回归拟合。 它们的 API 首先需要一个公式，和一个`DataFrame`作为第二个参数，`data`。 我们将函数和数据集关键字对`（sm.poisson，'data'）`传递给`pipe`：

In [7]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

bb = pd.read_csv('../../baseball.csv', index_col='id')

bb.head()

,Unnamed: 0,year,stint,team,lg,g,ab,r,h,X2b,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
id,,,,,,,,,,,,,,,,,,,,,
ansonca01,4,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN
forceda01,44,1871,1,WS3,NaN,32,162,45,45,9,...,29.0,8.0,0.0,4,0.0,NaN,NaN,NaN,NaN,NaN
mathebo01,68,1871,1,FW1,NaN,19,89,15,24,3,...,10.0,2.0,1.0,2,0.0,NaN,NaN,NaN,NaN,NaN
startjo01,99,1871,1,NY2,NaN,33,161,35,58,5,...,34.0,4.0,2.0,3,0.0,NaN,NaN,NaN,NaN,NaN
suttoez01,102,1871,1,CL1,NaN,29,128,35,45,3,...,23.0,3.0,1.0,1,0.0,NaN,NaN,NaN,NaN,NaN


In [8]:
(bb.query('h > 0')
 .assign(ln_h = lambda df: np.log(df.h))
 .pipe((sm.poisson, 'data'), 'hr ~ ln_h + year + g + C(lg)')
 .fit()
 .summary()
)

Optimization terminated successfully.
         Current function value: 2.948624
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                     hr   No. Observations:                18236
Model:                        Poisson   Df Residuals:                    18227
Method:                           MLE   Df Model:                            8
Date:                Fri, 05 Oct 2018   Pseudo R-squ.:                  0.5616
Time:                        09:16:38   Log-Likelihood:                -53771.
converged:                       True   LL-Null:                   -1.2265e+05
                                        LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -30.0945      0.219   -137.475      0.000     -30.524     -29.665
C(lg)[T.AL]     0.0118      0.052      0.228      0.819      -0.089       0.113
C(lg)[T.FL]    -0.1573      0.125     -1.254      0.210      -0.403       0.089
C(lg)[T.NL]    -0.0079      0.052     -0.153      0.878      -0.109       0.093
C(lg)[T.PL]     0.2712      0.096      2.830      0.005       0.083       0.459
C(lg)[T.UA]    -0.4598      0.381     -1.206      0.228      -1.207       0.288
ln_h            1.1199      0.012     95.898      0.000       1.097       1.143
year            0.0136      0.000    125.931      0.000       0.013       0.014
g               0.0028      0.000     13.533      0.000       0.002       0.003
===============================================================================
"""

`pipe()`方式可以使在pandas对象上连续执行多个函数操作的代码变得简洁明了，并且有助于跟踪函数及其相应参数的应用顺序。

## pipe语法
`DataFrame.pipe(func, *args, **kwargs)`

### Parameters:	
**func** : function

要应用于NDFrame的函数。 `args`和`kwargs`是传入`func`的参数。

注意：默认`func`的第一个位置参数是`NDFrame`，如果在定义`func`时，第一个位置参数不是`NDFrame`，在`pipe()`中需要使用`data`关键字引用`NDFrame`的名称，显示地指定`NDFrame`。

**args** : positional arguments passed into func. 位置参数，传递给`func`。

**kwargs** : a dictionary of keyword arguments passed into func. 字典关键字参数，传递给`func`。

### Returns:	
**object** : the return type of func. 

**对象**：`func`返回的类型。

**Notes**

当将期望在`Series`或`DataFrames`上将函数链接在一起时，使用`.pipe`，而不是这样写：

> `f(g(h(df), arg1=a), arg2=b, arg3=c)`

应该这样写：

```
>>> (df.pipe(h)
...    .pipe(g, arg1=a)
...    .pipe(f, arg2=b, arg3=c)
... )
```

如果有一个函数不是将数据作为第一个参数，例如作为第二个参数，则传递一个元组，指示哪个关键字需要数据。 例如，假设`f`将数据作为`arg2`：

```
>>> (df.pipe(h)
...    .pipe(g, arg1=a)
...    .pipe((f, 'arg2'), arg1=a, arg3=c)
...  )
```

即将 `func`参数和该函数需要数据集作为参数的参数名称字符串组成一个元组，作为`pipe()`的第一个参数。

## Example 1

假设想要将以下三个函数应用于DataFrame或Series：

- 第一个函数对数据集做减法
- 第二个函数对数据集做除法
- 第三个函数对数据集先做乘方，再做与标量求模。

先创建这些函数：

```
def adder(dataset,addn):
    dataset += addn
    return dataset
    
def div(dataset,divn):
    dataset /= divn
    return dataset
    
def time_mod(dataset, tn, mn):
    dataset = np.mod(np.power(dataset,mn),tn)
    return dataset
```

首先，我使用`adder`函数向数据集指定列的每个条目添加2。然后，使用`div`函数将指定列的每个条目除以2。最后，用`time_log`函数将指定列执行乘方和log运算。

## 示例代码

In [9]:
import numpy as np
import pandas as pd

In [10]:
def adder(dataset,addn):
    dataset += addn
    return dataset

In [11]:
def div(dataset,divn):
    dataset /= divn
    return dataset

In [12]:
def time_mod(dataset, tn, mn):
    dataframe = np.mod(np.power(dataset,mn),tn)
    return dataset

In [13]:
dn = np.arange(1,10,1).reshape(3,3)

In [14]:
df = pd.DataFrame(data = dn,index=['A','B','C'],columns=['col1','col2','col3'])

In [15]:
(df.pipe(adder, addn=2)
       .pipe(div, divn=4)
       .pipe(time_mod, tn=4, mn=2)
)

,col1,col2,col3
A,0.75,1.00,1.25
B,1.50,1.75,2.00
C,2.25,2.50,2.75


**注意**：

要应用`pipe()`，函数的第一个参数一般应是数据集，在`pipe()`链接函数时，函数的第一个参数省略，会自动传递调用的数据集给`func`。例如，`adder`接受两个参数`adder(dataframe, addn)`。第一个参数`dataframe`接收数据集，可以直接使用`pipe(adder, addn)`。

如果要应用于`pipe()`的函数的第一个参数不是数据集，就不起作用。解决办法是指定`pipe`函数中引用数据集的参数名称，即告知pipe，该函数中哪个参数引用数据集。方法是：

- 将 `func`名 和函数定义中代表数据集的参数名称字符串组成一个元组，作为`pipe()`的第一个参数传递。**参数名称需要引用包括，即将参数名称作为字典键**。

## 示例 2

现在重新定义`adder()`函数，将数据集作为第二个参数：`adder（addn，dataset）`。

要在`pipe()`中使用该函数，格式如下：`DataFrame.pipe((adder,"dataset"), addn)`。

In [16]:
def adder(addn,dataset):  #adder函数的dataset参数表示数据集
    dataset += addn
    return dataset

In [17]:
df = pd.DataFrame(data = dn,index=['A','B','C'],columns=['col1','col2','col3'])

In [18]:
(df.pipe((adder, 'dataset'),addn=2) #adder函数的dataset参数表示数据集，和函数名组成一个元组，参数名需要用引号包括。
       .pipe(div,divn=4)
       .pipe(time_mod,tn=4,mn=2)
)

,col1,col2,col3
A,0.75,1.00,1.25
B,1.50,1.75,2.00
C,2.25,2.50,2.75
